In [1]:
import pandas as pd
import re

In [2]:
# Load the CSV file released by Open Data on 09/26/2024
path = r'C:\Users\betebari\Documents\C2VSim_Texture\OSWCR'
uploaded_csv_path = path + '\GeologicLog_FreeForm.csv'
df = pd.read_csv(uploaded_csv_path)

<>:3: SyntaxWarning: invalid escape sequence '\G'
<>:3: SyntaxWarning: invalid escape sequence '\G'
C:\Users\betebari\AppData\Local\Temp\ipykernel_19832\1909367415.py:3: SyntaxWarning: invalid escape sequence '\G'
  uploaded_csv_path = path + '\GeologicLog_FreeForm.csv'
C:\Users\betebari\AppData\Local\Temp\ipykernel_19832\1909367415.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(uploaded_csv_path)


In [3]:
# Check if the 'DESCRIPTION' column exists
if 'DESCRIPTION' not in df.columns:
    raise KeyError("The 'DESCRIPTION' column does not exist in the DataFrame.")

In [4]:
# Define a dictionary for category conversions
conversions = {
    # Clay-related typos
    'clay': [
        'mud', 'adobe', 'mudflow', 'slay', 'nclay', 'c;lay', 'corcoran', 'clayh', 'kclay', 'clayy*', 'glay',
        'cclay', 'cllay', 'clsy', 'clayl', 'cay', 'c;au', 'clayclay', 'clay1', 'play', 'clcay', 'clat', 'caly', 'ckay',
        'chay', 'clayu', 'clay7', 'clayt', 'clayyyyy', 'clayq', 'c;ay', 'ctay', 'cla', 'chay', 'cloay','slays',
    ],
    # Gravel-related typos
        'gravel': [
        'grit', 'birds eye', 'graavel', 'gravvel', 'gravl', 'gravrel', 'garvel', 
        'crystalized minerals', 'colluvium', 'grave;', 'gravle', 'greavel', 'gavel', 'graval'
    ],
    
    'graywacke': [
        'sand stone', 'sandstone', 'ione', 'graywackey', 'gray wackey', 'sandstone,', 'sandstone;','sandsatone','sandtone',
        'greywacke', 'sandsstone', 'sandstrone', 'cementedsand', 'saandstone', 'sanstone', 'sd/stone','sandstonew',
    ],
    
    'cobble': ['grit cobble birds eye birds eye crystalized minerals colluvium','cobble',],
    'shale': ['shail','shalestone','shele','schel','shlae','scheil','shade','shake',],

    'decomposed granite': ['dg', 'd,g.', 'd/g', 'd.g', 'd & g', 'd.g.', 'dg', 'dgf', 'quartz', 'digi','quaartz','quart/8','dj',],
    'conglomerate': ['cemented gravel', 'cobblestone','gravelstone','pebblestone','cobble stone','cobble stones',],
    'soil': [
        'overburden', 'dirt', 'ground', 'overberdon', 'top soil', 'over burden', 'top seal', 
        'ovcerburden', 'overrburden', 'ovberburden', 'soill', 'soilo', 'overdurden','drit'
    ],
    'clayey sand': ['sand clay', 'sand and clay', 'sand-clay', 'sand,clay', 'sand&clay', 'clay loam'],
    'gravely sand': ['sand gravel', 'sand and gravel', 'sand & gravel'],
    'sandy clay': ['clay sand', 'clay sandy','sandyclay','sandclay',],
    'sandy gravel': ['gravel.sandy.', 'gravel and sand', 'alluvium'],
    'poorly graded sand': ['mixed sands'],
    'black sand': ['blacksand'],
    'coarse sand': ['sand-coarse','coarsesand',],
    'clayey gravel': ['gravel with clay'],
    'sand brown': ['sand-brown','brwnsand'],
    'clay brown': ['clay-brown'],
    'brown clay':['brownclay'],
    'sand black': ['sand-black'],
    'medium sand': ['mediumsand'],
    'fine sand': ['finesand'],
    'coarse': ['course', 'channel'],
    'ash': ['ashâ', 'gray ash'],
    'serpentine': ['serp.', 'serp','serpintine','surpentine',],
    'hardpan': [
        'hard pen', 'pann', 'hadpan', 'p an', 'paan', 'hard pan', 'hard pan,', 'hard pan;', 'hardapan',
        'hand pan', 'had pan', 'hard an'
    ],    
    'silty sand': ['silty.sandy', 'silty.sand',],
    'mehrton frm': ['merton'],
    'peat': ['peet'],
    'mudstone': ['stopped drilling','stop drilling', 'hard clay','claystone','rocki','bedrock','rock','mud stone',],
    'gray': ['grey'],
    'boulder': ['boulders'],
    'tuff': ['tuft'],
    'blue': ['bluish'],
    'quartz': ['quarts'],
    'organic soil': ['marsh deposits'],
    'silty clay': ['siltyclay'],
     # Sand-related typos
    'sand': [
        'sands', 'sands;', 'samd', 'sqands', 'saqnd', 'snad', 'sans', 'sande', 'sannd', 'sadn', 'sandw', 'sand245',
        'brnsand', 'sanc', 'sana', 'ssand', 'sand3', 'snd', 'sandsand', 'sabd', 'sandd', 'san d', 'dand',
    ],
    'clay' : ['clays',],
    'gravel': ['gravels'],
    'gravel clay': ['gravelclay'],
    'basalt': ['balsalt'],
    'silt' : ['silts','sult','sit'],
    'frac': [
        'fracture', 'ffract', 'frecture', 'fractures,', 'ffrac', 'fractures', 'fractures;',
        'fractured,', 'fractgure', 'frack', 'fractrue', 'fractureq', 'fract', 'fradctured', 'freacture,'
    ],    
    'brown black':['brownblack'],
    'cobble' : ['cobbles'],
    'granite':['grantie','gran.','grannite','granit','granite3','grranite','grainte','franite','granited'],
    'blue clay': ['clay-blue', 'clay blue','blueclay'], 
    'schist': ['shist'],
    'slate': ['salate'],
    'loam' : ['loom','lome'],
    'soapstone': ['soap sdtone'],
    'greenstone': ['greestone'],
    'limestone': ['lime stone'],
# Add more conversions as needed
}

In [5]:
# Function to apply conversions
def apply_conversions(text, conversions):
    if pd.isna(text):  # Handle NaN or None
        return text
    text = str(text)  # Ensure it's a string
    # Loop through the conversion dictionary
    for value, synonyms in conversions.items():
        for synonym in synonyms:
            # Replace synonyms with the correct value using word boundaries for whole-word replacement
            text = re.sub(r'\b{}\b'.format(re.escape(synonym)), value, text, flags=re.IGNORECASE)
    return text.lower()  # Convert the final result to lowercase

# Apply conversions to the DESCRIPTION column
df['DESCRIPTION'] = df['DESCRIPTION'].apply(lambda x: apply_conversions(x, conversions))

df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,DESCRIPTION
0,WCR2023-009618,150,170,blue clay
1,WCR2023-009618,170,225,sand
2,WCR2023-009618,225,235,brown clay
3,WCR2023-009618,235,260,sand
4,WCR2023-009618,260,275,gray clay


In [6]:
# Initialize NEW_DESCRIPTION column from DESCRIPTION
df['NEW_DESCRIPTION'] = df['DESCRIPTION'].copy()

# Define color qualifiers
color_qualifiers = [
    'black and white','b&w','red','redish', 'green','greenish', 'black','blackish', 'brown','brownish','gray', 'grayish',
    'yellow','yellowish', 'dark', 'light', 'tan', 'colored', 'blue','blueish','purple','purpleish', 'orange','orangeish',
    'olive', 'grey', 'purple-hued','multi-colored','brownish to grey','brn', 'salt & pepper', 'white','whitish','redish', 
]

# Define texture qualifiers
texture_qualifiers = [
    'loose', 'coarse', 'fine', 'compacted','crushed','med. to course', 'minor','medium', 'large','firm', 'small', 'grey',
    'minor', 'eroded','tight','broken','intermixed','mix','smooth','ball sized', 'packed', 'brittle', 'porous', 'pea', 
    'welded', 'mixed', 'softer', 'joint', 'chunky','large', 'big','solid','firm','heavy','very stiff','crystallized minerals',
    'laminated','poorly graded', 'well graded','grained','graded', 'shell', 'fine to coarse','fine to medium','hard',
    'fine-medium', 'choppy' ,'some', 'streaks of clay',  'layered', 'fairly smooth','cemented'
]

# Function to extract color qualifiers and remove them from the description
def extract_color_and_update_description(text):
    extracted_colors = []
    if isinstance(text, str):
        words = text.split()
        updated_words = [word for word in words if word not in color_qualifiers]
        extracted_colors = [word for word in words if word in color_qualifiers]
        return ' '.join(extracted_colors), ' '.join(updated_words)
    return 'none', text

# Function to extract texture qualifiers and remove them from the description
def extract_texture_and_update_description(text):
    extracted_textures = []
    if isinstance(text, str):
        words = text.split()
        updated_words = [word for word in words if word not in texture_qualifiers or word in ['fractured', 'weathered', 'decomposed']]
        extracted_textures = [word for word in words if word in texture_qualifiers and word not in ['fractured', 'weathered', 'decomposed']]
        return ','.join(extracted_textures), ' '.join(updated_words)
    return 'none', text

# Extract colors and update the description
df['COLOR QUALIFIER'], df['NEW_DESCRIPTION'] = zip(
    *df['NEW_DESCRIPTION'].apply(extract_color_and_update_description)
)

# Extract textures and update the description
df['TEXTURE QUALIFIER'], df['NEW_DESCRIPTION'] = zip(
    *df['NEW_DESCRIPTION'].apply(extract_texture_and_update_description)
)

# Display the updated DataFrame
df[['NEW_DESCRIPTION','COLOR QUALIFIER', 'TEXTURE QUALIFIER']].head()

,NEW_DESCRIPTION,COLOR QUALIFIER,TEXTURE QUALIFIER
0,clay,blue,
1,sand,,
2,clay,brown,
3,sand,,
4,clay,gray,


In [7]:
uscs_keywords_keep = {
    # Compound soil types
    'sand/clay', 'sand & clay', 'clay/sand', 'clay & sand', 'clay/silt', 'clay & silt',
    'gravel/sand', 'gravel & sand', 'sand/gravel', 'sand & gravel', 'silty gravel',
    'sand/gravel/clay', 'gravel/sand/silt', 'silty clay', 'decomposed granite', 'sand/silt', 
    'sandy silt', 'gravelly clay', 'gravelly sand', 'silty clayey sand', 'sandy silt clay', 'gravelly silt',
    'gravelly silty sand', 'clayey sand', 'silty sand', 'gravel/silt', 'organic gravel', 'gravelly peat',
    'gravel/clay', 'gravelly loam', 'sandy loam', 'clayey loam', 'silty loam', 'gravelly clayey sand', 
    'organic clay', 'organic silt', 'organic loam', 'peaty soil', 'peaty silt', 'sand/clay/silt',
    'clay loam', 'sandy clay loam', 'silt loam', 'silty gravel/sand', 'clay/gravel','clay/sand/gravel',
  
    # Sedimentary rocks
    'graywacke', 'conglomerate', 'shale', 'siltstone', 'limestone', 'cobblestone',  'mudstone', 
    
    # Soil classifications
    'silt', 'sand',  'gravel',  'clay', 'boulder','cobble', 

    # Soil descriptors
    'sticky', 'fat', 'lean', 'hardpan', 'organic', 'adobe',  'weathered','poorly graded sand', 'well graded sand',
    'poorly graded gravel', 'well graded gravel',
    
    # Rocks and minerals
    'basalt', 'basaltic', 'pumice', 'latite', 'volcanic', 'volcanics', 'cinder', 'ash', 
    'tufa', 'tuff', 'lava', 'rhyolite', 'granite', 'diorite', 'quartz', 'gabbro', 'quartzite', 
    'granodiorite', 'igneous', 'andesite', 'greenstone', 'slate', 'schist', 'serpentine', 
    'metasediment', 'phyllite', 'argillite', 'bluestone', 'soapstone', 'chert','fractured',
    'fractured rock', 'gouge', 'hardrock', 'rock','bedrock','frac','tuscan','lapilli','limestone',

    # Other geological terms
    'sediment', 'alluvium', 'loam', 'peat', 'organics','topsoil','soil',
}

In [8]:
# Select only the key words to convert into USCS
descrip = df['NEW_DESCRIPTION']

# Build the regex pattern
pattern = r'\b(?:' + '|'.join(map(re.escape, uscs_keywords_keep)) + r')\b'

# Apply the regex, handling NaN values by converting them to an empty string
extracted_words = [re.findall(pattern, str(d), re.IGNORECASE) for d in descrip]

# Add the extracted words as a new column
df['KEYWORDS_LIST'] = extracted_words

# Join the keywords into a single string
new_descriptions = df['KEYWORDS_LIST']
separator = ','
d = []

for new_description in new_descriptions:
    new_descrip = separator.join(new_description)
    d.append(new_descrip.lower())

# Add the final keywords column
df['KEYWORDS'] = d

# Drop the intermediate column
df = df.drop(['KEYWORDS_LIST'], axis=1)

# Display the updated DataFrame
df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,DESCRIPTION,NEW_DESCRIPTION,COLOR QUALIFIER,TEXTURE QUALIFIER,KEYWORDS
0,WCR2023-009618,150,170,blue clay,clay,blue,,clay
1,WCR2023-009618,170,225,sand,sand,,,sand
2,WCR2023-009618,225,235,brown clay,clay,brown,,clay
3,WCR2023-009618,235,260,sand,sand,,,sand
4,WCR2023-009618,260,275,gray clay,clay,gray,,clay


In [9]:
#Remove redundant keywords
def unique_list(l):
    ulist = []
    for x in l: 
        if x not in ulist: 
            ulist.append(x)
    return ulist

In [10]:
# Define USCS codes and their associated keywords
uscs_codes = [
    '(ML)', '(GP)', '(SP-SM)', '(GC)', '(CL)', '(CH)', '(MH)', '(GP-GM)', '(PT)',
    '(GP-GC)', 'GM', '(ml)','(OL)','(OL/OH)' 'SP-SM', '(SC-SM)', 'CH', '(SC)','(SW)','SC', 'MH',
    'SM-SC', 'SM-ML', 'SM-GP', 'SP-GP', 'SW-GW', 'SC-GC', 'SC-ML', 'SP/GP/CL', 'CL/SP',
    'GW-GM', 'OH/CH', 'CL/CH', 'Cl', 'SW-SM', 'SW-SC', 'SP-SM', 'SP-SC', 'OL', 'GW-GC', 'GP-SP',
    'GP-GM', 'GP-GC', 'GP-CG', 'CL-ML', 'CH+GP', 'SM/GM', 'sm/ml/sm/cl', 'sm/ml/sm/ml', 'sm/ml/sp/ml',
    'SM>ML>CL', 'SM>ML>SP>CL>SM', 'SM>ML,SP>SC>CL', 'PT.','PT', 'sm', 'sw', 'sp', 'ml', 'sc', 'cl', 'ch', 'mh', 'gp', 'gc',
    'sm/sp/ml', 'sp-sm', 'cl-ml', 'ch+gp', 'sm/ml', 'sm/sp', 'sm/sp/ml','(cl=50%, ml=30%)','(cl=55%, ml=35%)','(cl=60%, ml=30%)'
    ,'(gw=45%, sp=40%)','(gw=50%, sp=40%)', '(cl=50%, ml=40%)', '(gw=60%, sp=30%)',
]

#Identify the USCS keyterms in the description
descrip = df.DESCRIPTION.astype(str)
pattern = r'\b(?:' + '|'.join(map(re.escape, uscs_codes)) + r')\b'
extracted_uscs = [re.findall(pattern,d,re.IGNORECASE) for d in descrip]
df['USCS_keys']= extracted_uscs

keys = df['USCS_keys']
separator = ','
d = []
for key in keys:
    comb = separator.join(key)
    d.append(comb)
df.USCS_keys = d


In [11]:
# Updated dictionary to map keywords into USCS codes including terms from uscs_keywords_keep
uscs_keywords = { 
    # Compound soil types
    'sand/clay|sand & clay|clay/sand|clay & sand': 'CL-SP',
    'clay/silt|clay & silt': 'CL-ML',
    'gravel/sand|gravel & sand|sand/gravel|sand & gravel': 'SP-GP',
    'silty gravel': 'GM',
    'sand/gravel/clay|gravel/sand/silt|sand clay gravel|sand silt gravel': 'SP-GC',
    'silty clay': 'CL', 'decomposed granite': 'DGRNT', 'sand/silt|sandy silt': 'SM',
    'gravelly clay': 'CL', 'gravelly sand': 'SP-GP', 'silty clayey sand': 'SM-SC',
    'sandy silt clay': 'ML-CL', 'gravelly silt': 'GM', 'gravelly silty sand': 'GM',
    'clayey sand': 'SC', 'silty sand': 'SM', 'gravel/silt': 'GM', 'organic gravel': 'GM',
    'gravelly peat': 'GP-PT', 'gravel/clay': 'GC', 'gravelly loam': 'GM',
    'sandy loam': 'ML', 'clayey loam': 'ML', 'silty loam': 'ML', 'gravelly clayey sand': 'GC-SC',
    'organic clay': 'OH', 'organic silt': 'OL', 'organic loam': 'OL', 'peaty soil|peat': 'PT',
    'peaty silt': 'PT', 'sand/clay/silt': 'CL/SP/ML', 'clay loam': 'CL', 'sandy clay loam': 'CL',
    'silt loam': 'ML', 'silty gravel/sand': 'GM', 'clay/gravel': 'CL-GC', 'clay/sand/gravel': 'CL-SC-GC',

    # Sedimentary rocks
    'conglomerate|cobblestone': 'CONG',
    'graywacke|sandstone': 'SDST',
    'shale': 'SHLE',
    'claystone|mudstone': 'CLSN',
    'siltstone': 'STST',

    # Soil classifications
    'silt': 'ML', 'sand': 'SP', 'gravel': 'GP', 'clay': 'CL', 'boulder|cobble': 'COBL',

    # Soil descriptors
    'sticky clay': 'CH', 'fat clay': 'CH', 'lean clay': 'CL', 'hardpan': 'ML', 'organic': 'OH',
    'adobe': 'CH', 'weathered': 'SW-GW', 'poorly graded gravel': 'GP', 'well graded gravel': 'GW',
    'poorly graded sand': 'SP', 'well graded sand': 'SW',
    
    # Rocks and minerals
    'basalt|basaltic': 'BSLT',
    'pumice|lapilli|tuscan': 'VFRG',
    'volcanic|volcanics':'VOLC',
    'limestone|lime stone':'LMST',
    'cinder|ash|tufa|tuff' : 'TUFF',
    'lava': 'LAVA', 'granite|granodiorite': 'GRNT', 'diorite|quartz|gabbro|quartzite': 'IGNS',
    'rhyolite|andesite|latite|igneous': 'IGNS', 'greenstone|serpentine|phyllite|argillite': 'META',
    'metasediment': 'META', 'bluestone|soapstone': 'META', 'slate|schist': 'SCHT',
    'chert': 'SDST', 'fractured|fractured rock|gouge|hardrock|rock|bedrock|frac': 'FRAC',

    # Other geological terms
    'sediment|alluvium': 'GW-SW',
    'topsoil|soil': 'TPSL',
}

# Apply the USCS codes using the updated keyword matching
df['USCS'] = df['KEYWORDS'].replace(uscs_keywords, regex=True)

In [12]:
# Filter rows where 'USCS_keys' is not NaN and not an empty string
df_na = df[df['USCS_keys'].notna() & (df['USCS_keys'] != '')]

# Use .loc[] to explicitly assign values to the 'USCS' column
df_na.loc[:, 'USCS'] = df_na['USCS_keys']

# Update the original DataFrame with the modified 'df_na'
df.update(df_na)

# Identify and display any unmatched values
unmatched_values = df[df['USCS'].isna()]['USCS_keys'].unique()
print("Unmatched 'USCS' values:", unmatched_values)

# Display the updated DataFrame
df.head()

Unmatched 'USCS' values: []


,WCRNUMBER,INTERVALSTART,INTERVALEND,DESCRIPTION,NEW_DESCRIPTION,COLOR QUALIFIER,TEXTURE QUALIFIER,KEYWORDS,USCS_keys,USCS
0,WCR2023-009618,150,170,blue clay,clay,blue,,clay,,CL
1,WCR2023-009618,170,225,sand,sand,,,sand,,SP
2,WCR2023-009618,225,235,brown clay,clay,brown,,clay,,CL
3,WCR2023-009618,235,260,sand,sand,,,sand,,SP
4,WCR2023-009618,260,275,gray clay,clay,gray,,clay,,CL


In [13]:
# Function to remove duplicates in a comma-separated string and convert to upper case
def remove_duplicates(keywords):
    if isinstance(keywords, str):
        # Split by comma, remove duplicates while maintaining order, join back, and convert to uppercase
        unique_keywords = ','.join(dict.fromkeys(keywords.split(','))).upper()
        return unique_keywords
    return keywords

# Apply the remove_duplicates function to the 'USCS' column
df['USCS'] = df['USCS'].apply(remove_duplicates)

# Display the updated DataFrame
df.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,DESCRIPTION,NEW_DESCRIPTION,COLOR QUALIFIER,TEXTURE QUALIFIER,KEYWORDS,USCS_keys,USCS
0,WCR2023-009618,150,170,blue clay,clay,blue,,clay,,CL
1,WCR2023-009618,170,225,sand,sand,,,sand,,SP
2,WCR2023-009618,225,235,brown clay,clay,brown,,clay,,CL
3,WCR2023-009618,235,260,sand,sand,,,sand,,SP
4,WCR2023-009618,260,275,gray clay,clay,gray,,clay,,CL


In [14]:
#Clean dataset
df_new = df[['WCRNUMBER', 'INTERVALSTART', 'INTERVALEND', 'DESCRIPTION','NEW_DESCRIPTION','USCS','KEYWORDS','COLOR QUALIFIER','TEXTURE QUALIFIER']]
df_new.head()

,WCRNUMBER,INTERVALSTART,INTERVALEND,DESCRIPTION,NEW_DESCRIPTION,USCS,KEYWORDS,COLOR QUALIFIER,TEXTURE QUALIFIER
0,WCR2023-009618,150,170,blue clay,clay,CL,clay,blue,
1,WCR2023-009618,170,225,sand,sand,SP,sand,,
2,WCR2023-009618,225,235,brown clay,clay,CL,clay,brown,
3,WCR2023-009618,235,260,sand,sand,SP,sand,,
4,WCR2023-009618,260,275,gray clay,clay,CL,clay,gray,


In [15]:
#Merge well spatial data on 'WCRNUMBER'
df1 = pd.read_csv(path + "\\UPDATED_wellcompletionreports.csv")
merged_df = pd.merge(df1,df_new, on='WCRNUMBER', how='inner')
merged_df.head()

,WCRNUMBER,DECIMALLATITUDE,DECIMALLONGITUDE,Zland,X,Y,INTERVALSTART,INTERVALEND,DESCRIPTION,NEW_DESCRIPTION,USCS,KEYWORDS,COLOR QUALIFIER,TEXTURE QUALIFIER
0,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,0.0,20.0,sand with mudstone,sand with mudstone,"SP,CLSN","sand,mudstone",,
1,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,20.0,40.0,sand with gravel and mudstone,sand with gravel and mudstone,"SP,GP,CLSN","sand,gravel,mudstone",,
2,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,40.0,60.0,sand with clay and gravel,sand with clay and gravel,"SP,CL,GP","sand,clay,gravel",,
3,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,60.0,80.0,sand with clay and gravel,sand with clay and gravel,"SP,CL,GP","sand,clay,gravel",,
4,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,80.0,100.0,clayey gravel and sand,clayey gravel and sand,"GP,SP","gravel,sand",,


In [16]:
# Dictionary mapping unmatched USCS values to more sensible classifications
uscs_mapping = {
    'SP/CH': 'SP-CH',
    'CH & SP': 'SP-CH',
    'SP & CH': 'SP-CH',
    'GRAVELLY SP': 'SW',
    'GRAVELLY CH': 'CH-GC',
    'SP-GP/CH': 'SP-CH',
    'GP-GW & SP': 'GP-GW-SP',
    'SP & GP-GW': 'SP-GP-GW',
    'GRAVELLY SC': 'SC-GC',
    'CH & ML': 'CH-ML',
    'GRAVELLY SM': 'SM-GM',
    'ORGANICS': 'ML',
    'GP-GW/CH' : 'GP-GW,CH',
    'ORGANIC': 'ML',
    'ML CH': 'ML-CH',
    'GP-GW/ML': 'GP-GW-ML',
    'SILTY ML': 'ML',
    'GRAVELLY ML': 'ML-GM',
    'CL-SP,SP/CH': 'CL-SP , SP-CH',
    'LEAN,CL-SP': 'CL-SP',
    'SILTY GP-GW': 'GP-GW,ML',
    'GMY SP' : 'GM,SP',
    'CLEY SP': 'SC-SW',
    'SM CL' : 'SM-CL',
    'STICKY': 'CH',
    'LOAM' : 'ML',
    'VOLCS' : 'VOLC',
    'OHS': 'OH',
}

# Function to map unmatched USCS values to sensible classifications
def map_uscs_classifications(uscs_value):
    return uscs_mapping.get(uscs_value, uscs_value)  # Return the mapped value or the original if not found

# Assuming you have a DataFrame called df with a 'USCS' column
merged_df['USCS_mapped'] = merged_df['USCS'].apply(map_uscs_classifications)

# Now df['USCS_mapped'] contains the more sensible USCS classifications

In [17]:
import pandas as pd

# Load the provided Excel file
file_path = r'C:\Users\betebari\Documents\C2VSim_Texture\OSWCR\USCS-averageKxy-CoarseFractions.xlsx'
excel_data = pd.read_excel(file_path)

# Strip any leading/trailing spaces from 'Sediment/Rock Type' column in the Excel data
excel_data['Sediment/Rock Type'] = excel_data['Sediment/Rock Type'].str.strip()

# Create a dictionary mapping Soil Classification to Average Hydraulic Conductivity
hydraulic_conductivity_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Hydraulic Conductivity (ft/day)']))

# Create a dictionary mapping Soil Classification to Average Coarse Fraction
coarse_fraction_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Average Coarse Fraction (%)']))

# Create separate mappings for Specific Yield, Ss, and Kv based on the Excel data columns
specific_yield_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Specific Yield (%)']))
ss_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Ss (1/L)']))
kv_mapping = dict(zip(excel_data['Sediment/Rock Type'], excel_data['Avg Kv (ft/day)']))

# Ensure 'USCS_mapped' column exists in 'merged_df'
merged_df['USCS_mapped'] = merged_df['USCS_mapped'].str.strip()

# Remove parentheses from USCS values
merged_df['USCS_mapped'] = merged_df['USCS_mapped'].str.replace(r'[\(\)]', '', regex=True)

# Function to calculate the average or maximum for multiple USCS classifications
def aggregate_uscs_values(uscs_value, mapping, agg_func='average'):
    uscs_list = [item.strip() for item in uscs_value.split(',')]
    values = [mapping.get(uscs) for uscs in uscs_list if uscs in mapping and pd.notna(mapping.get(uscs))]
    
    # Handle missing values by returning a default value (12 for coarse fraction as per your code logic)
    if not values:
        return 12
    
    if values:
        if agg_func == 'average':
            return sum(values) / len(values)
        elif agg_func == 'max':
            return max(values)
    return None

# Apply the aggregation function for Hydraulic Conductivity and Coarse Fraction
merged_df['HydraulicConductivity'] = merged_df['USCS_mapped'].apply(lambda x: aggregate_uscs_values(x, hydraulic_conductivity_mapping, agg_func='average'))
merged_df['AverageCoarseFraction'] = merged_df['USCS_mapped'].apply(lambda x: aggregate_uscs_values(x, coarse_fraction_mapping, agg_func='average'))

# Apply the aggregation function for Specific Yield, Ss, and Kv using their own mappings
merged_df['Avg Specific Yield (%)'] = merged_df['USCS_mapped'].apply(lambda x: aggregate_uscs_values(x, specific_yield_mapping, agg_func='average'))
merged_df['Avg Ss (1/L)'] = merged_df['USCS_mapped'].apply(lambda x: aggregate_uscs_values(x, ss_mapping, agg_func='average'))
merged_df['Avg Kv (ft/day)'] = merged_df['USCS_mapped'].apply(lambda x: aggregate_uscs_values(x, kv_mapping, agg_func='average'))

# Identify and display any unmatched values
unmatched_values = merged_df[merged_df['HydraulicConductivity'].isna()]['USCS_mapped'].unique()
print("Unmatched 'USCS' values:", unmatched_values)

# Clean data by removing unnecessary columns if they exist
if 'Unnamed: 0' in merged_df.columns:
    merged_df = merged_df.drop(['Unnamed: 0'], axis=1)

# Drop the intermediate column 'USCS' if not needed
merged_df = merged_df.drop(['USCS'], axis=1, errors='ignore')  # Errors='ignore' avoids issues if the column doesn't exist

# Rename the 'USCS_mapped' back to 'USCS'
merged_df.rename(columns={'USCS_mapped': 'USCS'}, inplace=True)

# Display the first few rows of the merged dataframe to verify
merged_df.head()


Unmatched 'USCS' values: []


,WCRNUMBER,DECIMALLATITUDE,DECIMALLONGITUDE,Zland,X,Y,INTERVALSTART,INTERVALEND,DESCRIPTION,NEW_DESCRIPTION,KEYWORDS,COLOR QUALIFIER,TEXTURE QUALIFIER,USCS,HydraulicConductivity,AverageCoarseFraction,Avg Specific Yield (%),Avg Ss (1/L),Avg Kv (ft/day)
0,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,0.0,20.0,sand with mudstone,sand with mudstone,"sand,mudstone",,,"SP,CLSN",4.143750,35.000000,16.750000,0.000543,0.2525
1,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,20.0,40.0,sand with gravel and mudstone,sand with gravel and mudstone,"sand,gravel,mudstone",,,"SP,GP,CLSN",75.269067,53.333333,17.500000,0.000379,0.3350
2,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,40.0,60.0,sand with clay and gravel,sand with clay and gravel,"sand,clay,gravel",,,"SP,CL,GP",75.272701,51.666667,17.333333,0.001017,0.3335
3,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,60.0,80.0,sand with clay and gravel,sand with clay and gravel,"sand,clay,gravel",,,"SP,CL,GP",75.272701,51.666667,17.333333,0.001017,0.3335
4,WCR2018-004181,35.0174,-119.289,1069.953118,838644.742317,3.881271e+06,80.0,100.0,clayey gravel and sand,clayey gravel and sand,"gravel,sand",,,"GP,SP",112.900850,75.000000,24.500000,0.000275,0.5000


In [18]:
# Filter rows where 'USCS_mapped' is NaN or an empty string
blank_uscs_mapped = merged_df[merged_df['USCS'].isna() | (merged_df['USCS'] == '')]

# Step 2: Remove duplicates based on the 'NEW_DESCRIPTION' column
blank_uscs_mapped_unique_desc = blank_uscs_mapped.drop_duplicates(subset=['NEW_DESCRIPTION'])

# Save the result to a new Excel file
output_file_step2 = 'blank_uscs_mapped_unique_description.xlsx'
blank_uscs_mapped_unique_desc.to_excel(output_file_step2, index=False)

print(f"Rows with unique 'NEW_DESCRIPTION' exported to {output_file_step2}")


Rows with unique 'NEW_DESCRIPTION' exported to blank_uscs_mapped_unique_description.xlsx


In [19]:
# List of terms to drop
drop_terms = [
    'casing', 'aggregate', 'as above at', 'asphalt', 'existing well','unknown','see original well log',
    'existing', 'not logged', 'open hole', 'please see attached', 'same as above','see attached log',
    'see attached','no recovery',
]

# Create a regex pattern to match any of these terms (case insensitive)
pattern = '|'.join([re.escape(term) for term in drop_terms])

# Filter rows where 'NEW_DESCRIPTION' does not contain any of the terms
filtered_df = blank_uscs_mapped_unique_desc[~blank_uscs_mapped_unique_desc['NEW_DESCRIPTION'].str.contains(pattern, case=False, na=False)]

# Optionally save the filtered DataFrame to an Excel file
filtered_df.to_excel('filtered_uscs_mapped_unique_desc.xlsx', index=False)

# Display the filtered DataFrame to check results
filtered_df.head()

,WCRNUMBER,DECIMALLATITUDE,DECIMALLONGITUDE,Zland,X,Y,INTERVALSTART,INTERVALEND,DESCRIPTION,NEW_DESCRIPTION,KEYWORDS,COLOR QUALIFIER,TEXTURE QUALIFIER,USCS,HydraulicConductivity,AverageCoarseFraction,Avg Specific Yield (%),Avg Ss (1/L),Avg Kv (ft/day)
27,WCR2018-001994,35.0000,-119.000,1502.228395,865108.831125,3.880360e+06,0,30,conductor,conductor,,,,,12.0,12.0,12.0,12.0,12.0
1369,WCR2023-013881,35.0590,-119.093,506.4829558,856358.794472,3.886572e+06,0.0,30.0,concrete (pier wall),concrete (pier wall),,,,,12.0,12.0,12.0,12.0,12.0
1690,WCR0096836,35.0415,-118.967,659.4793518,867936.318657,3.885088e+06,329.0,357.0,hard drilling very rough,drilling very rough,,,hard,,12.0,12.0,12.0,12.0,12.0
2975,WCR2022-009877,35.1903,-118.914,351.6351819,872095.719370,3.901801e+06,0.0,107.0,see boring log for more information.,see boring log for more information.,,,,,12.0,12.0,12.0,12.0,12.0
3987,WCR2021-002915,35.2074,-118.887,393.7611675,874477.260653,3.903800e+06,757.0,788.0,brown,,,brown,,,12.0,12.0,12.0,12.0,12.0


In [20]:
# Save the updated DataFrame to a new CSV file
outpath = r'C:\Users\betebari\Documents\C2VSim_Texture\OSWCR'
output_file = outpath +'\\1-updated_geologiclog_freeform.csv'
merged_df.to_csv(output_file, index=None)

print(f"Final updated CSV file saved as '{output_file}'")

Final updated CSV file saved as 'C:\Users\betebari\Documents\C2VSim_Texture\OSWCR\1-updated_geologiclog_freeform.csv'
